In [1]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import gc
import os

In [2]:
# Load test_sup and find index
load_path = '/home/kai/data/kaggle/talkingdata/4thplace/'                     
print('sup read done')
with open('./log.txt', 'w') as f: f.write('start\n')

sup read done


In [3]:
load_path = '/home/kai/data/kaggle/talkingdata/4thplace/'
file_format = '{}.ftr'
train = pd.read_feather(load_path + '0.ftr')
    
print('train(downsampled) done')
val = pd.read_feather(load_path + 'val_cleaned.ftr')
print('val done')

train(downsampled) done
val done


In [4]:
import random as rn
import os

target = 'is_attributed'
os.environ['PYTHONHASHSEED'] = '0'
seed = 2018
np.random.seed(seed)
rn.seed(seed)
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_boost_round': 2000,
        'learning_rate': 0.04,
        'num_leaves': 76,
        'num_threads': 20, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': 7, # no limit. This is used to deal with over-fitting when #data is small.
        'min_child_samples': 20,
        'min_child_weight': 5,
        'min_split_gain': 0,
        'reg_alpha': 0,
        'reg_lambda': 0,
        'subsample': 1.0,
        'subsample_for_bin': 200000,
        'subsample_freq': 1,
        'bagging_seed': seed,
        'early_stopping_round':100,
        'verbose': 0,
        'scale_pos_weight': 32,#*0.00246100785192494,
        'metric' : [ 'auc'],
        'feature_fraction_seed': seed,
        'drop_seed': seed,
        'data_random_seed': seed,
        'max_bin': 188,
        'colsample_bytree': 0.45
    }

categorical_col = [ 'app', 'device', 'os', 'channel', 'hour', 'range_count_ip_day_hour']
feature_cols = list(train.columns.values)
remove_list = [
    'click_count_in_next_n_hour_ip_app_os_channel_1',
    'count_app_channel',
    'count_ip_day',
    'count_app_day_hourminute',
    'unique_ratio_day_ip_machine',
    'unique_ratio_day_ip_device',
    'unique_ratio_day_ip_app',
    'unique_ratio_machine_channel',
    'unique_ratio_day_ip_os',
    'unique_ratio_machine_app',
    'unique_ratio_machine_ip',
    'cumcount_ip_app_device_os_day_hour',
    'range_count_ip_channel_dayhourminute',
    'range_count_ip_day',
    'range_count_ip_device_os_dayhourminute',
    'range_count_ip_dayhourminute',
    'range_count_ip_dayhourminute10',
    'range_count_app_os_channel_dayhourminute',
    'com_ip','index', 'click_id', 'ip','click_time', 'is_test', 'is_attributed', 'day',
    'minute', 'minute10', 'hourminute', 'hourminute10', 'dayhourminute', 'dayhourminute10', 'machine', 'click_timestamp',
    'filter_time_to_n_next_click_2_day_ip_app_device_os',
    'filter_time_to_n_next_click_1_day_ip_app_device_os'
]
# remove_list.extend(['com1_ip', 'minute10', 'minute', 'hour', 'hourminute10', 'hourminute', 'dayhourminute', 'click_timestamp', 'index', 'dayhourminute10', 'day', 'count_ratio_ip_channel', 'machine', 'cumcount_ratio_ip_day', 'os', 'count_ratio_ip_machine', 'device', 'woe_ip_nextClickLeakDayFlt', 'nunique_machine_ip', 'unique_ratio_machine_channel', 'channel', 'woe_ip_device_nextClickLeakDayFlt', 'count_device_os_day_hourminute10', 'unique_ratio_machine_app', 'woe_device', 'count_ratio_channel_app', 'count_ratio_app_channel', 'count_ratio_machine_ip', 'nunique_machine_app', 'unique_ratio_machine_ip', 'nunique_machine_channel', 'woe_ip_channel_nextClickLeakDayFlt', 'com_ip', 'woe_ip_os_nextClickLeakDayFlt', 'woe_ip_channel', 'woe_ip', 'woe_ip_app', 'woe_ip_app_nextClickLeakDayFlt', 'range_count_app_os_channel_dayhourminute10', 'cumcount_ip_app_device_os_day_hour', 'woe_ip_device', 'woe_ip_os_channel', 'woe_ip_os', 'ip', 'woe_ip_device_os_channel', 'woe_ip_app_os_channel', 'woe_ip_app_device', 'woe_ip_device_channel', 'woe_ip_app_channel', 'woe_ip_device_os', 'range_count_app_os_channel_dayhourminute', 'nunique_day_ip_device', 'woe_ip_app_device_channel', 'woe_os', 'range_count_ip_day', 'woe_ip_app_os', 'nunique_day_ip_app', 'unique_ratio_day_ip_machine', 'variance_ip_device_hour', 'range_count_ip_dayhourminute10', 'filter_time_to_n_next_click_2_day_ip_app_device_os', 'count_app_os_channel_day_hour', 'range_count_ip_dayhourminute', 'unique_ratio_day_ip_channel', 'nunique_day_ip_channel', 'unique_ratio_day_ip_os', 'cumcount_ip_day', 'count_ip', 'count_ip_app_os_channel_day', 'unique_ratio_day_ip_device', 'unique_ratio_day_ip_app', 'nunique_day_ip_os', 'count_ip_day', 'nunique_day_ip_machine', 'woe_ip_app_device_os', 'range_count_ip_day_hour', 'range_count_ip_device_os_dayhourminute10', 'range_count_ip_device_os_dayhourminute', 'count_ip_app_device_os_day_hour', 'count_ip_os', 'count_ip_app_os_channel', 'app', 'filter_time_to_n_next_click_1_day_ip_app_device_os', 'cumcount_app_device_os_day', 'count_ip_device_os_day_hourminute10', 'woe_device_nextClickLeakDayFlt', 'time_to_n_next_click_2_day_ip_app_device_os'])
feature_cols = [x for x in feature_cols if x not in remove_list]
categorical_col = [x for x in categorical_col if x not in remove_list]
print(len(feature_cols))
# print(feature_cols)

83


In [5]:
print(feature_cols)

['app', 'device', 'os', 'channel', 'hour', 'woe_ip_app_channel', 'woe_ip_device_nextClickLeakDayFlt', 'nunique_machine_app', 'woe_os_channel_nextClickLeakDayFlt', 'variance_ip_device_hour', 'range_count_ip_device_os_dayhourminute10', 'woe_app_device_os_channel', 'woe_channel', 'count_ip_app_os_channel', 'nunique_day_ip_os', 'woe_app_nextClickLeakDayFlt', 'woe_ip_channel', 'nunique_machine_channel', 'time_to_n_next_click_1_day_ip_app_device_os', 'woe_os', 'woe_ip_device_os_channel', 'count_ip_app_device_day', 'com1_ip', 'count_ratio_ip_machine', 'count_ip_app_device_os_day_hour', 'count_app_device_day_hour', 'woe_app_device_os', 'woe_ip_app_os_channel', 'range_count_app_os_channel_dayhourminute10', 'woe_app_os_channel', 'unique_ratio_day_ip_channel', 'woe_ip', 'cumcount_ip_day', 'nunique_day_ip_channel', 'woe_ip_channel_nextClickLeakDayFlt', 'woe_ip_device', 'range_count_ip_channel_dayhourminute10', 'cumcount_app_device_os_day', 'count_ratio_app_channel', 'woe_ip_app_nextClickLeakDayFlt

In [6]:
test_supplement = pd.read_feather(load_path + 'test_cleaned.ftr')
train_len = train.shape[0]
val_len = val.shape[0]
df = pd.concat([train, val, test_supplement])
for x in train.columns.values:
    if x.split('_')[0] == 'woe':
        df[x] = (df[x] - df[x].mean()) / df[x].std()

train = df[:train_len]
val = df[train_len : train_len+val_len]
test_supplement = df[train_len+val_len:]

In [7]:
def train_lightgbm(x_train, x_val, feature_cols, categorical_feature, params, best_round = None, target='is_attributed'):
    param = params.copy()
    y_train = x_train[target].values
    y_val = x_val[target].values
    
    lgb_train = lgb.Dataset(x_train[feature_cols], y_train, categorical_feature = categorical_feature)
    lgb_val = lgb.Dataset(x_val[feature_cols], y_val, categorical_feature = categorical_feature)
    print('start training')
    model = lgb.train(param, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=10)
    return model


# Train model and get prediction

In [8]:
model = train_lightgbm(train, val, feature_cols, categorical_col, params)
with open('./log.txt', 'a') as f: f.write('train done\n')
print('prediction done')

start training


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 100 rounds.
[10]	valid_0's auc: 0.979134
[20]	valid_0's auc: 0.979841
[30]	valid_0's auc: 0.980249
[40]	valid_0's auc: 0.980675
[50]	valid_0's auc: 0.981585
[60]	valid_0's auc: 0.981827
[70]	valid_0's auc: 0.98238
[80]	valid_0's auc: 0.982773
[90]	valid_0's auc: 0.983696
[100]	valid_0's auc: 0.984448
[110]	valid_0's auc: 0.984621
[120]	valid_0's auc: 0.984833
[130]	valid_0's auc: 0.98499
[140]	valid_0's auc: 0.985031
[150]	valid_0's auc: 0.98514
[160]	valid_0's auc: 0.985218
[170]	valid_0's auc: 0.985295
[180]	valid_0's auc: 0.985333
[190]	valid_0's auc: 0.985308
[200]	valid_0's auc: 0.985329
[210]	valid_0's auc: 0.985369
[220]	valid_0's auc: 0.985396
[230]	valid_0's auc: 0.985377
[240]	valid_0's auc: 0.98538
[250]	valid_0's auc: 0.985391
[260]	valid_0's auc: 0.98543
[270]	valid_0's auc: 0.985433
[280]	valid_0's auc: 0.985448
[290]	valid_0's auc: 0.985418
[300]	valid_0's auc: 0.985427
[310]	valid_0's auc: 0.985434
[320]	valid_0's auc: 

In [9]:
with open('./log.txt', 'a') as f: f.write('test rading done\n')
test_index = np.load('/home/kai/data/kaggle/talkingdata/data/supplement2testInexFinal.npy')
test = test_supplement.iloc[test_index]
print(test.shape)
pred = model.predict(test[feature_cols])
with open('./log.txt', 'a') as f: f.write('test done\n')
print('done')

(18790469, 118)
done


# Submit result

In [10]:
sub_dict = {}
df_sub = pd.DataFrame()
test_clickid = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv', usecols=['click_id'])
with open('./log.txt', 'a') as f: f.write('test old done\n')
print('done')
df_sub['click_id'] = test_clickid['click_id']
df_sub['is_attributed'] = pred
print('predicting file done!')
with open('./log.txt', 'a') as f: f.write('saving\n')
df_sub.to_csv('/home/kai/data/kaggle/talkingdata/4thplace/b2.csv.gz', compression='gzip', index=False)
with open('./log.txt', 'a') as f: f.write('done\n')
print('done')

done
predicting file done!
done
